# Swap KCBS | Playfield 

## Libraries

In [1]:
import numpy as np
import sympy as sp
import sympy.physics.quantum as sq
import sympy.physics.quantum.qubit as sqq
import sympy.physics.quantum.gate as sqg

## Inputs

In [1]:
class inputs:
    graphSize = 5
    p = 1/np.sqrt(5)*np.array([1,1,1,1,1])

## The Sympy Part | playground

In [25]:
k0 = sq.Ket('φ')
k1 = sq.Ket('ψ')
superposition = k0+k1; superposition

|φ> + |ψ>

In [26]:
inner_prod = sq.Dagger(superposition)*superposition; inner_prod

(<φ| + <ψ|)*(|φ> + |ψ>)

In [27]:
sq.qapply(inner_prod)

<φ|φ> + <φ|ψ> + <ψ|φ> + <ψ|ψ>

In [28]:
density = sq.qapply(superposition*sq.Dagger(superposition)); density

|φ>*<φ| + |φ>*<ψ| + |ψ>*<φ| + |ψ>*<ψ|

In [31]:
qubit_state = sqq.Qubit('00'); qubit_state

|00>

In [33]:
isinstance(qubit_state,sq.State)

True

In [37]:
gate_operator = sqg.HadamardGate(0); gate_operator

H(0)

In [38]:
isinstance(gate_operator,sq.Operator)

True

In [3]:

a=sqq.Qubit(0)
b=sqq.Qubit(1)

d = sq.qapply(sq.Dagger(a)*b).doit(); d

0

### Playing with classes

In [50]:
class A:
    gla=1

class B:
    glaba=3
    
class C(B,A):
    glow=4

c=C()

print(isinstance(c,A),isinstance(c,B),isinstance(c,C))


True True True


### Playing with the ket object

In [118]:
class onKet(sq.Ket):
#     a=1
    
    @classmethod
    def _eval_innerproduct(self,bra,**hints):
        if label == bra.label:
            return sp.Integer(1)
        else:
            return sp.Integer(12)
    
    @classmethod
    def dual_class(self):
        return onBra
    
    def iAm(self):
        return self.label
    
class onBra(sq.Bra):
#     a=2

    @classmethod
    def _eval_innerproduct(self,ket,**hints):
        if self.label == ket.label:
            return sp.Integer(1)
        else:
            return sp.Integer(0)
    
    @classmethod
    def dual_class(self):
        return onKet

In [117]:
a=onKet(0)
b=onKet(1)
a.label,b.label
sq.Dagger(b).label
a.iAm

<bound method onKet.iAm of |0>>

In [96]:
c=a+b; c

|0> + |1>

In [97]:
sq.qapply(sq.Dagger(b)*b).doit()

12

In [62]:
d=sq.Dagger(b)*b
d.doit()

<1|1>